In [1]:
pip install opencv-python albumentations

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import os
import albumentations as alb
import time

# =============================================================================
# FUNCTION 1: INTERACTIVE WEBCAM REGISTRATION
# =============================================================================
def get_user_details():
    """
    Main function to get user's decision, details, and a confirmed photo
    via webcam.
    """
    print("\n--- Starting Interactive Webcam Registration ---")
    while True:
        enquiry = input("Do you want to take a picture or not? (Yes/No): ").lower()

        if enquiry in ["yes", "y", "ye", "es"]:
            while True:
                name = input("Enter your name: ")
                if not name:
                    print("No name given.")
                    continue
                if name.replace(' ', '').isalpha():
                    break
                else:
                    print("Give me a valid name.")

            while True:
                id = input("Enter your ID (No special characters are allowed): ")
                if not id:
                    print("No ID given.")
                    continue
                if id.isalnum():
                    break
                else:
                    print("Invalid input. Please enter only numbers and letters for your ID.")

            # --- Photo Capture and Augmentation Logic ---
            try:
                base_data_folder = "Data"
                user_folder_path = os.path.join(base_data_folder, name)
                os.makedirs(user_folder_path, exist_ok=True)

                print("\nStarting camera...")
                cap = cv2.VideoCapture(0)
                if not cap.isOpened():
                    print("Error: Could not open webcam.")
                else:
                    # Loop to allow for retaking the photo
                    while True:
                        print("Taking a picture in 3 seconds...")
                        time.sleep(1)
                        print("2...")
                        time.sleep(1)
                        print("1...")
                        time.sleep(1)

                        ret, frame = cap.read()
                        if not ret:
                            print("Error: Failed to capture frame.")
                            break

                        # --- NEW: Save a temporary preview and ask for confirmation ---
                        temp_image_path = "temp_preview.jpg"
                        cv2.imwrite(temp_image_path, frame)
                        print(f"Image captured. A preview has been saved as '{temp_image_path}'.")
                        print("Please open this file to check the image.")

                        confirmation = input("Would you like to keep this image? (Yes/No): ").lower()

                        if confirmation in ["yes", "y"]:
                            # User is happy with the image, proceed.
                            print("Image confirmed. Saving permanently and generating augmentations...")

                            # Define the final path and rename the temp file to save it
                            file_path = os.path.join(user_folder_path, f"{name}_{id}_photo.jpg")
                            os.rename(temp_image_path, file_path)
                            print(f"Original photo saved in folder: '{user_folder_path}'")

                            # --- Image Augmentation ---
                            print("Now generating 20 augmented images...")
                            augmentor = alb.Compose([
                                alb.HorizontalFlip(p=0.5),
                                alb.RandomBrightnessContrast(p=0.2),
                                alb.RandomGamma(p=0.2),
                                alb.RGBShift(p=0.2),
                            ])
                            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                            image_resized = image_rgb # Using your latest "no resize" logic

                            for i in range(20):
                                augmented = augmentor(image=image_resized)
                                augmented_image_rgb = augmented['image']
                                augmented_image_bgr = cv2.cvtColor(augmented_image_rgb, cv2.COLOR_RGB2BGR)
                                aug_file_path = os.path.join(user_folder_path, f"{name}_{id}_augmented_{i+1}.jpg")
                                cv2.imwrite(aug_file_path, augmented_image_bgr)

                            print(f"Successfully saved 20 augmented images in '{user_folder_path}'.")
                            break # Exit the retake loop

                        else:
                            # User wants to retake the picture
                            print("Discarding image. Let's try again.")
                            if os.path.exists(temp_image_path):
                                os.remove(temp_image_path)
                            # The loop will now repeat, taking a new picture.

                cap.release()

            except Exception as e:
                print(f"\nAn error occurred while trying to take or augment the photo: {e}")
                if 'cap' in locals() and cap.isOpened():
                    cap.release()

            print(f"\nThank you, {name}. Your details have been recorded.")
            break

        elif enquiry in ["no", "n", "o"]:
            print("Ok, proceeding without taking a picture.")
            break
        else:
            print("Invalid response. Please type 'Yes' or 'No'.")

# =============================================================================
# FUNCTION 2: AUTOMATED BATCH REGISTRATION
# =============================================================================
def register_images_from_folder(input_folder):
    """
    Loops through an input folder, reads each image, and then saves the
    original plus 20 augmented versions to a structured 'Data' folder.
    """
    print("\n--- Starting Automated Batch Registration ---")

    # Define the main data folder
    base_data_folder = "Data"

    # Check if the input folder exists
    if not os.path.isdir(input_folder):
        print(f"Error: Input folder '{input_folder}' not found.")
        return

    print(f"Starting batch registration from folder: '{input_folder}'")

    # Loop through every file in the specified input folder
    for filename in os.listdir(input_folder):

        # Check if the file is a common image type
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):

            # --- 1. Extract Name, ID, and Set Up Paths ---
            base_filename = os.path.splitext(filename)[0]
            if '_' in base_filename:
                parts = base_filename.rsplit('_', 1)
                if len(parts) == 2:
                    name, id = parts[0], parts[1]
                else:
                    print(f"Warning: Filename '{filename}' not in 'Name_ID' format. Using base filename as name.")
                    name = base_filename
                    id = "unknown" # Assign a default ID
            else:
                print(f"Warning: Filename '{filename}' not in 'Name_ID' format. Using base filename as name.")
                name = base_filename
                id = "unknown" # Assign a default ID

            print(f"  > Processing file: '{filename}' -> Name: '{name}', ID: '{id}'")
            user_folder_path = os.path.join(base_data_folder, name)
            os.makedirs(user_folder_path, exist_ok=True)
            original_image_path = os.path.join(input_folder, filename)

            # --- 2. Load and Save the Original Image ---
            try:
                frame = cv2.imread(original_image_path)
                if frame is None:
                    print(f"Warning: Could not read image '{filename}'. Skipping.")
                    continue

                file_path = os.path.join(user_folder_path, f"{name}_{id}_photo.jpg")
                cv2.imwrite(file_path, frame)
                print(f"\nSuccessfully registered '{name}' and saved original photo.")

                # --- 3. Image Augmentation ---
                print(f"Now generating 20 augmented images for '{name}'...")
                augmentor = alb.Compose([
                    alb.HorizontalFlip(p=0.5),
                    alb.RandomBrightnessContrast(p=0.2),
                    alb.RandomGamma(p=0.2),
                    alb.RGBShift(p=0.2),
                ])
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image_resized = image_rgb

                for i in range(20):
                    augmented = augmentor(image=image_resized)
                    augmented_image_rgb = augmented['image']
                    augmented_image_bgr = cv2.cvtColor(augmented_image_rgb, cv2.COLOR_RGB2BGR)
                    aug_file_path = os.path.join(user_folder_path, f"{name}_{id}_augmented_{i+1}.jpg")
                    cv2.imwrite(aug_file_path, augmented_image_bgr)

                print(f"Successfully saved 20 augmented images in '{user_folder_path}'.")

            except Exception as e:
                print(f"\nAn error occurred while processing '{filename}': {e}")
                continue # Skip to the next image

    print("\n--- Batch Registration Complete ---")

# =============================================================================
# SCRIPT EXECUTION (MAIN MENU)
# =============================================================================
if __name__ == "__main__":

    print("=======================================")
    print("  Face Registration System")
    print("=======================================")
    print("Please choose an option:")
    print("  1: Register a new user (via Webcam)")
    print("  2: Register existing images (from Folder)")
    print("=======================================")

    choice = input("Enter your choice (1 or 2): ")

    if choice == '1':
        get_user_details()

    elif choice == '2':
        # Define the name of the folder containing your source images
        INPUT_IMAGE_FOLDER = "Register"
        register_images_from_folder(INPUT_IMAGE_FOLDER)

    else:
        print("Invalid choice. Please run the script again and enter 1 or 2.")

    print("\nRegistration script finished.")


  Face Registration System
Please choose an option:
  1: Register a new user (via Webcam)
  2: Register existing images (from Folder)

--- Starting Interactive Webcam Registration ---
Ok, proceeding without taking a picture.

Registration script finished.


In [4]:
# =============================================================================
# CELL 1: Imports and Initial Setup
# =============================================================================
# Purpose: Import all necessary libraries and configure the environment.
# -----------------------------------------------------------------------------
import os
import cv2
import numpy as np
import warnings

# Suppress unnecessary warnings for a cleaner output
warnings.filterwarnings("ignore")
print("Cell 1/4: Libraries imported and environment configured.")


Cell 1/4: Libraries imported and environment configured.


In [5]:
# =============================================================================
# CELL 2: Model File Verification
# =============================================================================
# Purpose: Define the model filenames and verify that they exist in the
#          project folder before proceeding.
# -----------------------------------------------------------------------------
# --- Model Information ---
FACE_DETECTION_MODEL = "face_detection_yunet_2023mar.onnx"
FACE_RECOGNITION_MODEL = "face_recognition_sface_2021dec.onnx"

def check_model_exists(model_name):
    """Checks if a model file exists in the current directory."""
    if not os.path.exists(model_name):
        print(f"Error: Model file not found: '{model_name}'.")
        print("Please make sure it is in the same folder as this script.")
        return False
    else:
        print(f"Model file '{model_name}' found.")
        return True

# --- Execute the file check for both models ---
detector_ready = check_model_exists(FACE_DETECTION_MODEL)
recognizer_ready = check_model_exists(FACE_RECOGNITION_MODEL)
print("Cell 2/4: Model file check complete.")


Model file 'face_detection_yunet_2023mar.onnx' found.
Model file 'face_recognition_sface_2021dec.onnx' found.
Cell 2/4: Model file check complete.


In [6]:

# =============================================================================
# CELL 3: Embedding Generation Function
# =============================================================================
# Purpose: Define the main function that iterates through a folder of images,
#          detects faces, and generates embeddings for each one.
# -----------------------------------------------------------------------------
def generate_face_embeddings(image_folder_path, detector, recognizer):
    """
    Detects faces in all images in a folder, generates embeddings, and saves them.
    """
    if not os.path.isdir(image_folder_path):
        print(f"Error: Source image folder not found at '{image_folder_path}'")
        return

    person_name = os.path.basename(image_folder_path)
    all_embeddings = []

    image_files = [f for f in os.listdir(image_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"\nFound {len(image_files)} images for '{person_name}'. Processing...")

    for filename in image_files:
        image_path = os.path.join(image_folder_path, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"  - Warning: Could not read '{filename}'. Skipping.")
            continue

        h, w, _ = image.shape
        detector.setInputSize((w, h))
        _, faces = detector.detect(image)

        if faces is not None and len(faces) > 0:
            try:
                face_coords = faces[0]
                aligned_face = recognizer.alignCrop(image, face_coords)
                embedding = recognizer.feature(aligned_face)
                all_embeddings.append(embedding)
                print(f"  - Generated embedding for {filename}")
            except Exception as e:
                print(f"  - Warning: Could not process face in '{filename}'. Error: {e}")
        else:
            print(f"  - Warning: No face detected in {filename}. Skipping.")

    # --- Save the Results ---
    if all_embeddings:
        output_dir = os.path.join("embeddings", person_name)
        os.makedirs(output_dir, exist_ok=True)

        embeddings_path = os.path.join(output_dir, f"{person_name}_embeddings.npy")
        labels_path = os.path.join(output_dir, f"{person_name}_labels.npy")

        labels = [person_name] * len(all_embeddings)

        np.save(embeddings_path, np.array(all_embeddings))
        np.save(labels_path, np.array(labels))

        print(f"\nSuccessfully saved {len(all_embeddings)} new embeddings for '{person_name}'.")
    else:
        print("\nNo new embeddings were generated.")

print("Cell 3/4: Embedding generation function defined.")

Cell 3/4: Embedding generation function defined.


In [7]:
# =============================================================================
# CELL 4: Execution Block
# =============================================================================
# Purpose: Load the models and call the main processing function for each
#          person's subfolder found within the main 'Data' directory.
# -----------------------------------------------------------------------------
if detector_ready and recognizer_ready:
    print("\nLoading models...")
    try:
        # Load Face Detector
        face_detector = cv2.FaceDetectorYN.create(
            model=FACE_DETECTION_MODEL, config="", input_size=(320, 320)
        )
        # Load Face Recognizer
        face_recognizer = cv2.FaceRecognizerSF.create(
            model=FACE_RECOGNITION_MODEL, config=""
        )
        print("Models loaded successfully.")

        # --- THE FIX IS HERE ---
        # The script now looks inside the 'Data' folder for subfolders to process.
        base_image_folder = "Data"
        if not os.path.isdir(base_image_folder):
             print(f"Error: The main '{base_image_folder}' directory was not found.")
        else:
            # Loop through each subfolder in the 'Data' directory (e.g., 'Lalith')
            for person_name in os.listdir(base_image_folder):
                person_folder_path = os.path.join(base_image_folder, person_name)
                if os.path.isdir(person_folder_path):
                    # Call the main function to run the process for each person
                    generate_face_embeddings(person_folder_path, face_detector, face_recognizer)

    except Exception as e:
        print(f"An error occurred while loading the models or running the process: {e}")
else:
    print("\nSkipping execution because one or both models are not available.")

print("\nCell 4/4: Execution finished.")




Loading models...
Models loaded successfully.

Found 21 images for 'Aaron_Eckhart'. Processing...
  - Generated embedding for Aaron_Eckhart_0001_augmented_1.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_10.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_11.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_12.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_13.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_14.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_15.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_16.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_17.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_18.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_19.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_2.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_20.jpg
  - Generated embedding for Aaron_Eckhart_0001_augmented_3.jpg
  - Gene

In [8]:
# =============================================================================
# CELL 1: Imports and Initial Setup
# =============================================================================
# Purpose: Import all necessary libraries and configure the environment.
# -----------------------------------------------------------------------------
import os
import cv2
import numpy as np
import warnings
import uuid
from datetime import datetime  # <-- ADDED for timestamping

# Suppress unnecessary warnings for a cleaner output
warnings.filterwarnings("ignore")
print("Cell 1/4: Libraries imported and environment configured.")


# =============================================================================
# CELL 2: Model and Known Embeddings Loader
# =============================================================================
# Purpose: Verify model files exist and load all previously generated embeddings
#          from the 'embeddings' directory into memory.
# -----------------------------------------------------------------------------
# --- Model Information ---
FACE_DETECTION_MODEL = "face_detection_yunet_2023mar.onnx"
FACE_RECOGNITION_MODEL = "face_recognition_sface_2021dec.onnx"

def check_model_exists(model_name):
    """Checks if a model file exists in the current directory."""
    if not os.path.exists(model_name):
        print(f"Error: Model file not found: '{model_name}'.")
        return False
    print(f"Model file '{model_name}' found.")
    return True

def load_known_faces(embeddings_base_folder):
    """Loads all saved embeddings and their corresponding labels."""
    known_embeddings = []
    known_labels = []
    if not os.path.isdir(embeddings_base_folder):
        print(f"Warning: Embeddings directory not found at '{embeddings_base_folder}'.")
        return [], []

    for person_name in os.listdir(embeddings_base_folder):
        person_dir = os.path.join(embeddings_base_folder, person_name)
        if os.path.isdir(person_dir):
            embeddings_path = os.path.join(person_dir, f"{person_name}_embeddings.npy")
            if os.path.exists(embeddings_path):
                embeddings = np.load(embeddings_path)
                labels = [person_name] * len(embeddings)
                known_embeddings.extend(embeddings)
                known_labels.extend(labels)
                print(f"- Loaded {len(embeddings)} embeddings for '{person_name}'.")

    return known_embeddings, known_labels

# --- Execute checks and loaders ---
detector_ready = check_model_exists(FACE_DETECTION_MODEL)
recognizer_ready = check_model_exists(FACE_RECOGNITION_MODEL)

known_face_embeddings, known_face_labels = [], []
if detector_ready and recognizer_ready:
    print("\nLoading known face embeddings...")
    known_face_embeddings, known_face_labels = load_known_faces("embeddings")
    if not known_face_embeddings:
        print("Could not find any pre-saved embeddings to compare against.")

print("Cell 2/4: Loaders finished.")


# =============================================================================
# CELL 3: Recognition Function
# =============================================================================
# Purpose: Define the function that captures a single image, generates an
#          embedding, and compares it against the known dataset.
# -----------------------------------------------------------------------------

# --- NEW: Helper function to save failed images ---
def save_false_case(image_frame, reason):
    """Saves an image that failed recognition to the 'False_Cases' folder."""
    try:
        false_cases_folder = "False_Cases"
        os.makedirs(false_cases_folder, exist_ok=True)

        # Generate a timestamp (e.g., "20251024_115501")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Create a unique filename
        false_filename = f"false_case_{reason}_{timestamp}.jpg"
        false_image_path = os.path.join(false_cases_folder, false_filename)

        # Save the image
        cv2.imwrite(false_image_path, image_frame)
        print(f"  - Saved unrecognized image to '{false_image_path}'")
    except Exception as e:
        print(f"  - Warning: Could not save false case image. Error: {e}")


def recognize_face_from_capture(detector, recognizer, known_embeddings, known_labels):
    """
    Captures a single image, generates a face embedding, compares it against
    the known database, and prints the result.
    """
    # --- Matching Threshold ---
    COSINE_THRESHOLD = 0.40

    # --- Capture a single frame ---
    print("\nOpening webcam for capture...")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print("Error: Failed to capture frame from webcam.")
        return

    print("Image captured successfully. Processing for recognition...")

    # --- Detect Face ---
    h, w, _ = frame.shape
    detector.setInputSize((w, h))
    _, faces = detector.detect(frame)

    if faces is None or len(faces) == 0:
        print("\n--- RESULT: Face Not Matched (Reason: No face detected in the image) ---")
        save_false_case(frame, "NoFaceDetected")  # <-- ADDED
        return

    # --- Generate Embedding for the detected face ---
    try:
        face_coords = faces[0]
        aligned_face = recognizer.alignCrop(frame, face_coords)
        new_embedding = recognizer.feature(aligned_face)

        best_match_name = "Unknown"
        best_match_score = 0

        # --- Compare with Known Faces ---
        for i, known_embedding in enumerate(known_embeddings):
            score = recognizer.match(new_embedding, known_embedding, cv2.FaceRecognizerSF_FR_COSINE)
            if score > best_match_score:
                best_match_score = score
                best_match_name = known_labels[i]

        # --- Determine and Print the Final Result ---
        if best_match_score >= COSINE_THRESHOLD:
            print(f"\n--- RESULT: Face Matched: {best_match_name} (Confidence Score: {best_match_score:.2f}) ---")

            try:
                # Define the main data folder and the user-specific subfolder for saving the image.
                base_data_folder = "Data"
                image_dir = os.path.join(base_data_folder, best_match_name)
                output_dir = os.path.join("embeddings", best_match_name)
                os.makedirs(image_dir, exist_ok=True)

                # Save the captured image with a unique name in the correct path.
                img_name = os.path.join(image_dir, f"{best_match_name}_{uuid.uuid1()}.jpg")
                cv2.imwrite(img_name, frame)
                print(f"  - Saved new reference image to '{img_name}'")

                # Load existing data, append new embedding, and save back
                embeddings_path = os.path.join(output_dir, f"{best_match_name}_embeddings.npy")
                labels_path = os.path.join(output_dir, f"{best_match_name}_labels.npy")

                if os.path.exists(embeddings_path):
                    person_embeddings = list(np.load(embeddings_path))
                    person_labels = list(np.load(labels_path))
                else:
                    person_embeddings, person_labels = [], []

                person_embeddings.append(new_embedding)
                person_labels.append(best_match_name)

                np.save(embeddings_path, np.array(person_embeddings))
                np.save(labels_path, np.array(person_labels))

                print(f"  - Updated embeddings for '{best_match_name}'. Total is now {len(person_embeddings)}.")
            except Exception as e:
                print(f"  - Warning: Could not save new data. Error: {e}")

        else:
            print(f"\n--- RESULT: Face Not Matched (Closest match was {best_match_name}, but score {best_match_score:.2f} was below threshold {COSINE_THRESHOLD}) ---")
            save_false_case(frame, f"FailedMatch_{best_match_name}")  # <-- ADDED

    except Exception as e:
        print(f"An error occurred during recognition: {e}")

print("Cell 3/4: Recognition function defined.")


# =============================================================================
# CELL 4: Execution Block
# =============================================================================
# Purpose: Load the models and call the main recognition function.
# -----------------------------------------------------------------------------
if detector_ready and recognizer_ready: # <-- MODIFIED (Removed check for known_face_embeddings)
    print("\nLoading models for recognition...")
    try:
        # Load Face Detector
        face_detector = cv2.FaceDetectorYN.create(
            model=FACE_DETECTION_MODEL, config="", input_size=(320, 320)
        )

        # Load Face Recognizer
        face_recognizer = cv2.FaceRecognizerSF.create(
            model=FACE_RECOGNITION_MODEL, config=""

        )
        print("Models loaded successfully.")

        # Check if embeddings are available *after* loading models
        if not known_face_embeddings:
            print("\nWarning: No pre-saved embeddings were found.")
            print("Running in 'detection only' mode. All faces will be logged as 'Unknown'.")

        # Run the recognition process
        recognize_face_from_capture(face_detector, face_recognizer, known_face_embeddings, known_face_labels)

    except Exception as e:
        print(f"A critical error occurred: {e}")
else:
    print("\nSkipping execution due to missing models.") # <-- MODIFIED

print("\nCell 4/4: Execution finished.")



Cell 1/4: Libraries imported and environment configured.
Model file 'face_detection_yunet_2023mar.onnx' found.
Model file 'face_recognition_sface_2021dec.onnx' found.

Loading known face embeddings...
- Loaded 22 embeddings for 'Aavula Anusha'.
- Loaded 23 embeddings for 'Ayush'.
- Loaded 32 embeddings for 'Lalith'.
- Loaded 22 embeddings for 'nagaraju'.
- Loaded 23 embeddings for 'nithin'.
- Loaded 22 embeddings for 'rajeev'.
- Loaded 24 embeddings for 'stanley'.
- Loaded 22 embeddings for 'vikas'.
- Loaded 22 embeddings for 'vishal'.
Cell 2/4: Loaders finished.
Cell 3/4: Recognition function defined.

Loading models for recognition...
Models loaded successfully.

Opening webcam for capture...
Image captured successfully. Processing for recognition...

--- RESULT: Face Not Matched (Reason: No face detected in the image) ---
  - Saved unrecognized image to 'False_Cases\false_case_NoFaceDetected_20251024_115852.jpg'

Cell 4/4: Execution finished.
